In [1]:
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential  
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient  

from azure.search.documents.models import (
    VectorizableTextQuery, 
)

import os
from dotenv import load_dotenv  
load_dotenv(override=True)

endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
service_name = os.getenv("AZURE_SEARCH_SERVICE")
api_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
DIMENSIONS = 1536
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_SERVICE = os.getenv('AZURE_OPENAI_SERVICE')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,  
    api_version="2023-10-01-preview",
    azure_endpoint = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com/"
)
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  
credential = AzureKeyCredential(key)


In [3]:

def vector_text_search_indx(user_query:str,index_name="bytewax-index", top_results = 3):  
    query_vec = VectorizableTextQuery(text=user_query, k=3, fields="vector", exhaustive=True)  
    search_client = SearchClient(os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT'), index_name, credential=credential)
    search_results = search_client.search(      
        search_text=None,      
        vector_queries= [query_vec],    
        select=["id", "meta", "content"],    
        top=top_results    
    )    
    output_str = ''    
    for result in search_results:      
        # print(f"Parent ID: {result['parent_id']}")      
        # print(f"Chunk: {result['chunk']}")      
        # print(f"Score: {result['@search.score']}")      
        # print(f"Title: {result['title']}")   
        # output_str += f"Parent ID: {result['parent_id']}\nChunk: {result['chunk']}\nScore: {result['@search.score']}\nTitle: {result['title']}\n\n"  
        output_str=f"{result['content']}"  
    return output_str
  
def get_ans(user_query):
    a=vector_text_search_indx(user_query,index_name='bytewax-index', top_results = 3)
    messages=[  
    {"role": "assistant", "content": "You are an AI assistant that helps in giving answers based on the provided data."},  
    {"role": "user", "content": 'Answer the query: {user_query} based on the below context:'+'/n'+vector_text_search_indx(user_query,index_name='bytewax-index', top_results = 3)}  
    ] 
    response = client.chat.completions.create(
        model="bytewax-workshop-gpt35",
        messages = messages,
        temperature=0,
        max_tokens=1000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )
    ans=response.choices[0].message.content
    # Print the AI-generated response
    # print(f"{query_results}")
    return ans

get_ans("Provide a summary of the information in your database")

"I'm sorry, but I cannot provide an answer to the query as the provided context does not contain any specific question or query. If you have a specific question related to the provided context, please feel free to ask and I'll be happy to help!"

In [ ]:
{"T": "n", "id": 39063802, "headline": "Piper Sandler Maintains Overweight on Virtus Inv, Raises Price Target to $267", "summary": "Piper Sandler  analyst Crispin Love   maintains Virtus Inv (NYSE:VRTS) with a Overweight and raises the price target from $265 to $267.", "author": "Benzinga Newsdesk", "created_at": "2024-05-29T13:26:51Z", "updated_at": "2024-05-29T13:26:52Z", "url": "https://www.benzinga.com/news/24/05/39063802/piper-sandler-maintains-overweight-on-virtus-inv-raises-price-target-to-267", "content": "Piper Sandler  analyst Crispin Love   maintains Virtus Inv (NYSE:<a class=\"ticker\" href=\"https://www.benzinga.com/stock/VRTS#NYSE\">VRTS</a>) with a Overweight and raises the price target from $265 to $267.", "symbols": ["VRTS"], "source": "benzinga"}
